<a href="https://colab.research.google.com/github/dbengp/dio-ml-sistema-de-recomenda-o-de-imagens-digital/blob/main/Sistema_de_Recomenda%C3%A7%C3%A3o_por_Imagens_Digitais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Recomendação por Imagens - DIO

1. Configura-se o ambiente no Colab

In [ ]:
!pip install tensorflow keras numpy matplotlib

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

2. Prossegue-se com o carregamento e pré-processamento dos Dados

In [ ]:
# Carregar o dataset CIFAR-10 (ou Fashion MNIST)
from tensorflow.keras.datasets import cifar10

# Carregar e dividir os dados
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalizar as imagens (valores entre 0 e 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Exibir algumas imagens
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_train[i])
    plt.axis('off')
plt.show()

3. Prossegue-se com a extração de características com uma CNN Pré-treinada

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

embedding_model = Model(inputs=base_model.input, outputs=base_model.output)

# Função para gerar embeddings
def get_embedding(img):
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    embedding = embedding_model.predict(img)
    return embedding.flatten()

train_embeddings = np.array([get_embedding(img) for img in x_train])

4.Treina-se o Modelo de Similaridade

In [ ]:
neighbors_model = NearestNeighbors(n_neighbors=5, metric='cosine')
neighbors_model.fit(train_embeddings)

# Função para encontrar imagens similares
def find_similar_images(query_img, top_k=5):
    query_embedding = get_embedding(query_img)
    distances, indices = neighbors_model.kneighbors([query_embedding], n_neighbors=top_k)
    return distances, indices

query_img = x_test[0]
distances, indices = find_similar_images(query_img)

# Exibir a imagem de consulta
plt.figure(figsize=(15, 5))
plt.subplot(1, 6, 1)
plt.imshow(query_img)
plt.title("Consulta")
plt.axis('off')

for i, idx in enumerate(indices[0]):
    plt.subplot(1, 6, i + 2)
    plt.imshow(x_train[idx])
    plt.title(f"Similar {i+1}")
    plt.axis('off')
plt.show()

5. Segue-se com a avaliação do Sistema

In [ ]:
query_img = x_test[10]
distances, indices = find_similar_images(query_img)

plt.figure(figsize=(15, 5))
plt.subplot(1, 6, 1)
plt.imshow(query_img)
plt.title("Consulta")
plt.axis('off')

for i, idx in enumerate(indices[0]):
    plt.subplot(1, 6, i + 2)
    plt.imshow(x_train[idx])
    plt.title(f"Similar {i+1}")
    plt.axis('off')
plt.show()